## This notebook has following parts:
#### PART A - Convert SCR model OCI to a AWS SageMaker compatible OCI.
#### PART B - Deploy OCI to SageMaker for inference  - create sagemaker model, endpoint configuration, endpoint
#### PART C - Inference/score the model and delete resources.
#### PART D - Delete resources created. Very important if you do not need them.

#### This notebook needs other files from same directory where you downloaded this notebook (DockerFile, sagemaker_server.py and launch_two_servers.sh) 

# ======================= Seperator =========================

#### Pre-requisites
##### 1. Docker setup on machine where you are running this notebook. We need this to build a new docker image (NO need for ROOT Access). 
##### 2. AWS CLI setup on machine where you are running this notebook. We need this to pull and push images from AWS ECR
##### 3. AWS profile setup for the user running this note book under /HOME/.aws/config.User should have IAM permissions to pull/push images to ECR and create resources on sagemaker. - AmazonElasticContainerRegistry, AmazonEC2ContainerRegistryFullAccess, AmazonSageMakerFullAccess*
##### 4. AWS SDK with in the python environment that this notebook access to. 

# ======================= Seperator =========================

# PART A - Convert SCR model OCI to a AWS SageMaker compatible OCI.

In [115]:
# set variables in this cell for PART A
DOCKER_CMD = "docker -H tcp://docker-server.company.com:2375"
SCR_PATH_ON_AWS_ECR = "1234567890.dkr.ecr.us-east-1.amazonaws.com/sunallscrtree1"
SCR_MODEL_NAME = SCR_PATH_ON_AWS_ECR.split("/")[1]

# following is the ouput. SageMaker compatible OCI. We named it same as SCR model with -sm suffuix
SM_SCR_MODEL_NAME = SCR_MODEL_NAME + "-sm"

# AWS 
AWS_REGION = "us-east-1"
AWS_ECR = "1234567890.dkr.ecr.us-east-1.amazonaws.com"  

import pprint
pp = pprint.PrettyPrinter(indent=1)
# pp.pprint(SCR_MODEL_NAME)


In [116]:
# Replace modelname, modelpath variables in code segments. We need to update 2 files Dockerfile and sagemaker_server.py 
# We will get mentioned files as part of git download (same way as you got this notebook).

# Dockerfile changes. Look for REPLACE_KEY_1 and change the immediate line  after that. This makes it idempotent. 
# use a different limiter to address / in filepaths. here we used # instead of / in second statement.
!sed -i "/REPLACE_KEY_1/{n;s/.*/FROM SCR_PATH_ON_AWS_ECR_LITERAL/}" Dockerfile
!sed -i "s#SCR_PATH_ON_AWS_ECR_LITERAL#$SCR_PATH_ON_AWS_ECR#" Dockerfile

# sagemaker_server.py changes. Look for REPLACE_KEY_2 and change the immediate line  after that. This makes it idempotent.
!sed -i "/REPLACE_KEY_2/{n;s/.*/SCR_MODEL_NAME = "SCR_MODEL_NAME_LITERAL"/}" sagemaker_server.py
!sed -i "s#SCR_MODEL_NAME_LITERAL#'$SCR_MODEL_NAME'#" sagemaker_server.py



In [117]:
# This cell takes care of login to AWS ECR and creating repo for the new SM compatible model.

# 1. Do a docker login first to AWS ECR so we can pull and push images
!/usr/local/bin/aws ecr get-login-password --region {AWS_REGION} | {DOCKER_CMD} login {AWS_ECR} --username AWS --password-stdin

pp.pprint(SCR_MODEL_NAME)
pp.pprint(SM_SCR_MODEL_NAME)

# 2. Create a REPO. Ignore errors if they say it already exists. 
!/usr/local/bin/aws ecr create-repository --repository-name {SM_SCR_MODEL_NAME} --region {AWS_REGION}


WARNING! Your password will be stored unencrypted in /xxxx/home/userid/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
'sunallscrtree1'
'sunallscrtree1-sm'
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:1234567890:repository/sunallscrtree1-sm",
        "registryId": "1234567890",
        "repositoryName": "sunallscrtree1-sm",
        "repositoryUri": "1234567890.dkr.ecr.us-east-1.amazonaws.com/sunallscrtree1-sm",
        "createdAt": "2022-03-23T22:27:34-04:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [118]:
# This cell Build a new Docker image compatible with sagemaker
# Dockerfile refers to all files required to create the new OCI. 

# 1. Build the image
!{DOCKER_CMD} build -t {SM_SCR_MODEL_NAME} .

# 2. Tag and push the new image back to AWS ECR REPO from step2
!{DOCKER_CMD} tag {SM_SCR_MODEL_NAME} {AWS_ECR}/{SM_SCR_MODEL_NAME}
!{DOCKER_CMD} push  {AWS_ECR}/{SM_SCR_MODEL_NAME}



Sending build context to Docker daemon  43.52kB
Step 1/12 : FROM 1234567890.dkr.ecr.us-east-1.amazonaws.com/sunallscrtree1
 ---> cd005f3142d9
Step 2/12 : USER root
 ---> Using cache
 ---> 4240bf17021e
Step 3/12 : RUN /bin/bash -c 'mkdir sagemaker ; mkdir /opt/tmp; chmod -R 777 /opt/tmp'
 ---> Using cache
 ---> 74dec3d8e0d8
Step 4/12 : WORKDIR /sagemaker
 ---> Using cache
 ---> de84b8060add
Step 5/12 : RUN /bin/bash -c 'python -m ensurepip --upgrade; pip3 install flask gunicorn urllib3'
 ---> Using cache
 ---> 08f024f56716
Step 6/12 : ADD sagemaker_server.py sagemaker_server.py
 ---> Using cache
 ---> 3ae655cf794a
Step 7/12 : ADD launch_two_servers.sh launch_two_servers.sh
 ---> Using cache
 ---> 60ec45222d4d
Step 8/12 : RUN chmod a+x launch_two_servers.sh
 ---> Using cache
 ---> 9a62d5da87c7
Step 9/12 : USER 1001:0
 ---> Using cache
 ---> 0af16603b43e
Step 10/12 : EXPOSE 8080
 ---> Using cache
 ---> 384077d2e7ff
Step 11/12 : EXPOSE 9090
 ---> Using cache
 ---> b5b84cd74b8a
Step 12/12 :

In [119]:
# View images we just pushed - You can do this from AWS console as well..
!/usr/local/bin/aws ecr list-images --repository-name {SM_SCR_MODEL_NAME}

{
    "imageIds": [
        {
            "imageDigest": "sha256:6a7d9c37ae9223f8a66991f8d0813f2fc5fee0aa8f3051e2be36000dab317174",
            "imageTag": "latest"
        }
    ]
}


# PART B - Deploy SAS model to SageMaker runtime 

In [120]:
# settings for PART-B

# following role to be created on AWS IAM screen with sagemaker-full-access. 
sagemaker_iam_role = 'arn:aws:iam::1234567890:role/service-role/AmazonSageMaker-ExecutionRole-20220320T163511'
sagemaker_inference_instance_type="ml.t2.medium"
sagemaker_inference_instance_count=1

# endpoint names arrived based on model name to keep it simple. 
sagemaker_endpoint_config_name = SM_SCR_MODEL_NAME + "-epconfig"
sagemaker_endpoint_name = SM_SCR_MODEL_NAME + "-ep"

import json

In [121]:
# create sagemaker session objects

import boto3

# If you want to use specific keys you can set them as well. I am just uing default $HOME/.aws/config,credentials file 
boto3_session = boto3.Session()
sagemaker_client = boto3_session.client('sagemaker')

In [124]:
# create all sagemaker assets required .

#Create sagemaker model with reference to SCR container we created for sagemaker.
create_model_response = sagemaker_client.create_model(
    ModelName = SM_SCR_MODEL_NAME,
    ExecutionRoleArn = sagemaker_iam_role,
    PrimaryContainer = {
        'Image': AWS_ECR + "/" + SM_SCR_MODEL_NAME
        
    })
pp.pprint(create_model_response)

# create end-point configuration
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=sagemaker_endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": SM_SCR_MODEL_NAME, 
            "InstanceType": sagemaker_inference_instance_type, # Specify the compute instance type.
            "InitialInstanceCount": sagemaker_inference_instance_count # Number of instances to launch initially.
        }
    ]
)
pp.pprint(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

## create endpoint
create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=sagemaker_endpoint_name, 
                                            EndpointConfigName=sagemaker_endpoint_config_name) 
print(create_endpoint_response["EndpointArn"])

{'ModelArn': 'arn:aws:sagemaker:us-east-1:1234567890:model/sunallscrtree1-sm',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '79',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Thu, 24 Mar 2022 02:29:11 GMT',
                                      'x-amzn-requestid': 'b030b39f-446d-4ccd-b788-ca0ea482e44d'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b030b39f-446d-4ccd-b788-ca0ea482e44d',
                      'RetryAttempts': 0}}
('Created EndpointConfig: '
 'arn:aws:sagemaker:us-east-1:1234567890:endpoint-config/sunallscrtree1-sm-epconfig')
arn:aws:sagemaker:us-east-1:1234567890:endpoint/sunallscrtree1-sm-ep


In [125]:
# Wait till the end point is active. We need this inservice before we can make predictions. 
resp = sagemaker_client.describe_endpoint(EndpointName=sagemaker_endpoint_name)
status = resp["EndpointStatus"]
pp.pprint("Status: " + status)

sagemaker_client.get_waiter("endpoint_in_service").wait(EndpointName=sagemaker_endpoint_name)

'Status: Creating'


# PART C - Invoke prediction on SCR model with sagemaker endpoint.

In [106]:
# define payload for prediction
payload = json.dumps({
    "inputs": [ 
{"name": "CLAGE", "value": 101.4660019},
{"name": "CLNO", "value": 8 },
{"name": "DEBTINC","value": 37.11361356},
{"name": "DELINQ","value": 0},
{"name": "DEROG","value": 0},
{"name": "JOB","value": "Other"},
{"name": "LOAN","value": 1700},
{"name": "MORTDUE","value": 30548},
{"name": "NINQ","value": 1},
{"name": "REASON","value": "HomeImp"},
{"name": "VALUE","value": 40320},
{"name": "YOJ","value": 9}
]})

In [107]:
# create runtime client and invoke prediction. In real life you would probably read an input file and convert them into json objects to invoke prediction services
# 
runtime_client = boto3_session.client("runtime.sagemaker")

response = runtime_client.invoke_endpoint(
    EndpointName=sagemaker_endpoint_name, ContentType="application/json", Body=payload
)
result = response["Body"].read().decode("ascii")

pp.pprint(result)

('{"version":1,"metadata":{"module_id":"sunallscrtree1","step_id":"score","timestamp":"2022-03-24T01:25:03.729Z","elapsed_nanos":"1551121"},"outputs":[{"name":"EM_CLASSIFICATION","value":"           '
 '0"},{"name":"EM_EVENTPROBABILITY","value":0.05627306273062},{"name":"EM_PROBABILITY","value":0.94372693726937},{"name":"I_BAD","value":"           '
 '0"},{"name":"P_BAD0","value":0.94372693726937},{"name":"P_BAD1","value":0.05627306273062},{"name":"_leaf_id_","value":8.0}]}')


# PART D - Delete resoruces (leave ECR OCI images for now.  You can delete them on AWS console ECR window). 

In [126]:
# delete resources in reverse order.

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=sagemaker_endpoint_name) 
# Delete endpoint configuration
sagemaker_client.delete_endpoint_config(EndpointConfigName=sagemaker_endpoint_config_name)
# Delete endpoint configuration
sagemaker_client.delete_model(ModelName=SM_SCR_MODEL_NAME)



{'ResponseMetadata': {'RequestId': '59415937-239f-4221-880b-1698d35d20e7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '59415937-239f-4221-880b-1698d35d20e7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 24 Mar 2022 03:10:52 GMT'},
  'RetryAttempts': 0}}